In [169]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

tf.compat.v1.disable_eager_execution()

In [170]:
df=pd.read_csv('sonar data.csv',header=None)
print(len(df.columns))

61


In [171]:
df.head() # Each number represents the energy within a particular frequency band

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [173]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [174]:
df.isnull().values.any()

False

In [175]:
X=df[df.columns[0:60]].values
y=df[df.columns[60]]

In [176]:
X

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

In [177]:
y

0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

In [178]:
y.value_counts()

M    111
R     97
Name: 60, dtype: int64

In [179]:
le=LabelEncoder()
y=le.fit_transform(y)

In [180]:
y=np.array(y)
y=y.reshape(-1,1)
ohe=OneHotEncoder()
y=ohe.fit_transform(y).toarray()

In [182]:
y[:5]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [183]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [184]:
X_train.shape

(166, 60)

In [185]:
X_test.shape

(42, 60)

In [186]:
learning_rate=0.01
training_epochs=100
cost_history=np.empty(shape=[1],dtype='float')

In [187]:
n_class=2 # Rock and Mine
x=tf.compat.v1.placeholder(tf.float32,[None,60])
w=tf.Variable(tf.zeros([60,n_class]))
b=tf.Variable(tf.zeros([n_class]))

In [188]:
# initialize all variable
# init=tf.global_variables_initializer()
init=tf.compat.v1.global_variables_initializer()

In [189]:
y=tf.compat.v1.placeholder(tf.float32,[None,n_class]) # actual y
pred=tf.nn.softmax(tf.matmul(x,w)+b) #predicted y

In [190]:
cost_function=tf.reduce_mean(-tf.reduce_sum((y*tf.math.log(pred)))) # claculating rmse

In [191]:
# tf.compat.v1.disable_v2_behavior()
training_step=tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [192]:
# initialize the session
sess=tf.compat.v1.Session()
sess.run(init)
mse_history=[]

In [193]:
# calculate the cost for each epoch
for i in range(training_epochs):
    sess.run(training_step,feed_dict=({x:X_train,y:y_train}))
    cost=sess.run(cost_function,feed_dict={x:X_train,y:y_train})
    cost_history=np.append(cost_history,cost)
    # after above 3 lines wights get updated and once we have updated weights we will evaluate the test data set.
    pred_y=sess.run(pred,feed_dict={x:X_test})
    print('epoch : ',i,' - ','cost: ',cost) # printing cost or error associated with each epoch
    # we want to calculate the mean squared error now
    mse=tf.reduce_mean(tf.square(pred_y-y_test)) # y_test is the actual output and pred_y is the predicted output
    mse_history.append(sess.run(mse))

epoch :  0  -  cost:  117.10058
epoch :  1  -  cost:  297.08432
epoch :  2  -  cost:  857.899
epoch :  3  -  cost:  253.21394
epoch :  4  -  cost:  838.73193
epoch :  5  -  cost:  253.43782
epoch :  6  -  cost:  820.8214
epoch :  7  -  cost:  253.01492
epoch :  8  -  cost:  799.34644
epoch :  9  -  cost:  256.6348
epoch :  10  -  cost:  776.8245
epoch :  11  -  cost:  261.6653
epoch :  12  -  cost:  752.83057
epoch :  13  -  cost:  268.38766
epoch :  14  -  cost:  727.9612
epoch :  15  -  cost:  276.00473
epoch :  16  -  cost:  702.3457
epoch :  17  -  cost:  284.0862
epoch :  18  -  cost:  676.3038
epoch :  19  -  cost:  291.92877
epoch :  20  -  cost:  650.1423
epoch :  21  -  cost:  298.9125
epoch :  22  -  cost:  624.2649
epoch :  23  -  cost:  304.65262
epoch :  24  -  cost:  599.0927
epoch :  25  -  cost:  309.1291
epoch :  26  -  cost:  574.98737
epoch :  27  -  cost:  312.50693
epoch :  28  -  cost:  552.1724
epoch :  29  -  cost:  314.89877
epoch :  30  -  cost:  530.71924
epo